In [2]:
import pandas as pd
import matplotlib.pyplot as plt
#%matplotlib inline
train_df = pd.read_csv("dsjtzs_txfz_training.txt",header=None,delim_whitespace=True)
train_df.columns = ["id","points_str","target_point","label"]


train_df.set_index(["id"],inplace=True)

def f(s):
    s =str(s)
    return s[0:-1]
train_df["points_str"]= train_df["points_str"].apply(f)


def  point_sort(x,y):
    if (x[2]>y[2]):
        return 1
    elif (x[2]<y[2]):
        return -1
    else:
        return 0
def f(t):
    s=str(t["points_str"])
    splited=s.split(";")
    ret = []
    for item in splited:
        item_splited = item.split(",")
        item_tuple = ( int(item_splited[0]),  int(item_splited[1]) ,int(item_splited[2]) )
        ret.append(item_tuple)
    sorted_arr = sorted(ret,point_sort)
    return sorted_arr
train_df["points_sorted_list"]=train_df[["points_str"]].apply(f,axis=1)

def f(s):
    sorted_arr=s["points_sorted_list"]
    _sum_x =0 
    _sum_y=0 
    _count =0
    ret=[]
    for i in range(0,len(sorted_arr)-1):
        if (sorted_arr[i][2]!=sorted_arr[i+1][2]):
                if _count==0:
                    ret.append(sorted_arr[i])
                else:
                    _count +=1
                    _sum_x+=sorted_arr[i][0]
                    _sum_y+=sorted_arr[i][1]
                    average_x =float( _sum_x ) / _count
                    average_y =float(_sum_y)/_count
                    ret.append((average_x,average_y,sorted_arr[i][2]))
                    _count =0
                    _sum_x=0
                    _sum_y=0
        else: 
            _sum_x += sorted_arr[i][0]
            _sum_y+=sorted_arr[i][1]
            _count+=1
    
    index = len(sorted_arr)-1
    if (sorted_arr[index][2] == sorted_arr[index-1][2]):
        _count+=1
        _sum_x+=sorted_arr[index][0]
        _sum_y+=sorted_arr[index][1]
        average_x =float( _sum_x ) / _count
        average_y =float(_sum_y)/_count
        ret.append((average_x,average_y,sorted_arr[index][2]))
    else:
        ret.append(sorted_arr[index])
    return ret

train_df["points_diff_time_list"]=train_df[["points_sorted_list"]].apply(f,axis=1)

# print train_df.loc[1,"points_str"]
# print "-----"
# print train_df.loc[1,"points_sorted_list"]

# points 都有数据
#print pd.isnull(train_df["points"]).value_counts()

# 2600 个1 正例；400个0 负例 (id:2601-3000)
#train_df.label.value_counts()

# id 都是不同的
#train_df.id.value_counts().shape[0]

In [3]:
def f_x(s):
     
    ret =[]
    for item in s:
        ret.append(int(item[0]))
    return ret
        
train_df["points_x"]= train_df["points_diff_time_list"].apply(f_x)

def f_y (s):
     
    ret=[]
    for item in s:
        ret.append(int(item[1]))
    return ret
train_df["points_y"]=train_df["points_diff_time_list"].apply(f_y)

def f_time (s):
  
    ret=[]
    for item in s:
        ret.append(int(item[2]) )
    return ret
train_df["points_time"]=train_df["points_diff_time_list"].apply(f_time)

def f_target_point_x(s):
    s = str(s)
    splited = s.split(",")
    return float(splited[0])
train_df["target_point_x"]=train_df["target_point"].apply(f_target_point_x)

def f_target_point_y (s):
    s =str(s)
    splited =s.split(",")
    return float(splited[1])
train_df["target_point_y"]=train_df["target_point"].apply(f_target_point_y)



train_df =train_df[["points_x","points_y","points_time","target_point_x","target_point_y","target_point","label"]]

#去掉points就１个的, id有：493,2019　(points没有２个的，就不用删除２个的了)
def filter(s):
    if len(s)==1:
        return False
    return True
filters = train_df["points_x"].apply(filter)
train_df=train_df[filters]


print train_df.shape

(2998, 7)


In [20]:
# plt.figure(figsize=(8,7),dpi=98)
# p1 = plt.subplot(211)
# p2 = plt.subplot(212)
# x = train_df.loc[1,"points_x"]
# y  = train_df.loc[1,"points_y"]
# target_x = train_df.loc[1,"target_point_x"]
# target_y = train_df.loc[1,"target_point_y"]
# print x
# print y
# p1.plot(x,y,'-',color='r',label="person")
# p1.plot(target_x,target_y,marker='*')
# p1.legend()
# plt.show()
ids = [4]
for id in ids:
    plt.figure(id)  #一个Figure对象可以包含多个子图（Axes）．指定了plt.figure(i)之后的操作就是在图表i 下进行的
    x = train_df.loc[id,"points_x"]
    y  = train_df.loc[id,"points_y"]
    target_x = train_df.loc[id,"target_point_x"]
    target_y = train_df.loc[id,"target_point_y"]
#     print x
#     print y
    print id
    print (target_x,target_y)
    print "-----------"
    if train_df.loc[id,"label"]==1:
        labelStr="person"
    else:
        labelStr="robort"
    plt.scatter(x,y,c="r",marker="*")
#  plt.plot(x,y,'-',color='r',label=labelStr)
    plt.plot(target_x,target_y,marker='*')
    plt.grid(True)
    plt.legend()
plt.show()

4
(675.0, 202.0)
-----------


In [7]:
def  f (s):
    ret = []
    points_x= s["points_x"]
    points_time =s["points_time"]
    for x in range(1,len(points_x)):
        velocity = (float(points_x[x])-points_x[x-1])/(float(points_time[x])-points_time[x-1])
        ret.append(velocity)
    return ret
train_df["velocity_x"]= train_df[["points_x","points_time"]].apply(f,axis=1)


def f(s):
    ret =[]
    points_y = s["points_y"]
    points_time = s["points_time"]
    for x in range(1,len(points_y)):
        velocity = (float(points_y[x])-points_y[x-1])/(float(points_time[x])-points_time[x-1])
        ret.append(velocity)
    return ret
train_df["velocity_y"]=train_df[["points_y","points_time"]].apply(f,axis=1)

def f(s):
    ret =[]
    velocity_x =s["velocity_x"]
    times = s["points_time"]
    if len(velocity_x)>1:
        for x in range (1,len (velocity_x)):
            accelerate_x = (float(velocity_x[x])- velocity_x[x-1])/((float(times[x+1])+times[x])/2 - (float(times[x])+times[x-1])/2)
            ret.append(accelerate_x)
    return ret
train_df["accelerate_x"]=train_df[["velocity_x","points_time"]].apply(f,axis=1)


def f(s):
    ret = []
    velocity_y =s["velocity_y"]
    times=s["points_time"]
    if (len(velocity_y)>1):
        for x in range(1,len(velocity_y)):
            accelerate_y = (float(velocity_y[x])-velocity_y[x-1])/((float(times[x+1])+times[x])/2 - (float(times[x])+times[x-1])/2)
            ret.append(accelerate_y)
    return ret
train_df["accelerate_y"]=train_df[["velocity_y","points_time"]].apply(f,axis=1)




In [21]:
import numpy as np
# # 对于point就一个点的，速度就是０
# train_df["velocity_x"]=train_df["velocity_x"].apply(lambda x: [0] if (len(x)==0) else x )
# train_df["velocity_y"]=train_df["velocity_y"].apply(lambda x: [0] if(len(x)==0) else x)

# #对于速度就一个值的，加速度为０
# train_df["accelerate_x"]=train_df["accelerate_x"].apply(lambda x: [0] if (len(x)==0) else x)
# train_df["accelerate_y"]=train_df["accelerate_y"].apply(lambda x: [0] if (len(x)==0) else x)

# points y 的方差，points x 的方差
train_df["points_y_var"]=train_df["points_y"].apply(lambda x :np.var(x))
train_df["points_x_var"]=train_df["points_x"].apply(lambda x:np.var(x))

# 均值：速度　加速度
train_df["velocity_x_mean"]=train_df["velocity_x"].apply(lambda x: np.mean(np.array(x)))
train_df["velocity_y_mean"]=train_df["velocity_y"].apply(lambda x:np.mean(np.array(x)))
train_df["accelerate_x_mean"]=train_df["accelerate_x"].apply(lambda x:np.mean(np.array(x)))
train_df["accelerate_y_mean"]=train_df["accelerate_y"] .apply(lambda x:np.mean(np.array(x)))

# 方差：速度　加速度
train_df["velocity_x_var"]=train_df["velocity_x"].apply(lambda x:np.var(np.array(x)))
train_df["velocity_y_var"]=train_df["velocity_y"].apply(lambda x:np.var(np.array(x)))
train_df["accelerate_x_var"]=train_df["accelerate_x"].apply(lambda x:np.var(np.array(x)))
train_df["accelerate_y_var"]=train_df["accelerate_y"].apply(lambda x:np.var(np.array(x)))

# 最大值: 速度 加速度
train_df["max_velocity_x"] =train_df["velocity_x"].apply(lambda x: max(x))
train_df["max_velocity_y"] = train_df["velocity_y"].apply(lambda x:max(x))
train_df["max_accelerate_x"] = train_df["accelerate_x"].apply(lambda x:max(x))
train_df["max_accelerate_y"] = train_df["accelerate_y"].apply(lambda x:max(x))

# 最小值:速度 加速度
train_df["min_velocity_x"]= train_df["velocity_x"].apply(lambda x: min(x))
train_df["min_velocity_y"]=train_df["velocity_y"].apply(lambda x:min(x))
train_df["min_accelerate_x"]=train_df["accelerate_x"].apply(lambda x:min(x))
train_df["min_accelerate_y"]=train_df["accelerate_y"].apply(lambda x:min(x))

# count : 速度  (即count:加速度，count:路径中每个点的y 距离目标y的 距离 )
train_df["count_velocity"]= train_df["velocity_x"].apply(lambda x: len(x))
 


#25%: 速度　加速度
train_df["velocity_x_percentile_25"]=train_df["velocity_x"].apply(lambda x :np.percentile(x,25))
train_df["velocity_y_percentile_25"]=train_df["velocity_y"].apply(lambda x:np.percentile(x,25))
train_df["accelerate_x_percentile_25"]=train_df["accelerate_x"].apply(lambda x:np.percentile(x,25))
train_df["accelerate_y_percentile_25"]=train_df["accelerate_y"].apply(lambda x:np.percentile(x,25))

# 75% : 速度　加速度
train_df["velocity_x_percentile_75"]=train_df["velocity_x"].apply(lambda x:np.percentile(x,75))
train_df["velocity_y_percentile_75"]=train_df["velocity_y"].apply(lambda x:np.percentile(x,75))
train_df["accelerate_x_percentile_75"]=train_df["accelerate_x"].apply(lambda x:np.percentile(x,75))
train_df["accelerate_y_percentile_75"]=train_df["accelerate_y"].apply(lambda x:np.percentile(x,75))

# 50% :速度方差
train_df["velocity_x_percentile_50"]=train_df["velocity_x"].apply(lambda x:np.percentile(x,75))
train_df["velocity_y_percentile_50"]=train_df["velocity_y"].apply(lambda x:np.percentile(x,75))
train_df["accelerate_x_percentile_50"]=train_df["accelerate_x"].apply(lambda x:np.percentile(x,75))
train_df["accelerate_y_percentile_50"]=train_df["accelerate_y"].apply(lambda x:np.percentile(x,75))


train_df["x_mean"]=train_df["points_x"].apply(lambda x:np.mean(np.array(x)))
train_df["y_mean"]=train_df["points_y"].apply(lambda x:np.mean(np.array(x)))

# 路径中x 的平均值距离目标x的距离
def f(s):
    x_mean =s["x_mean"]
    target_point_x = s["target_point_x"]
    return x_mean-target_point_x
train_df["x_mean_dis_target_x"] = train_df[["x_mean","target_point_x"]].apply(f,axis=1)

# 路径中　y的平均值距离目标y的距离　
def f(s):
    y_mean =s["y_mean"]
    target_point_y = s["target_point_y"]
    return y_mean - target_point_y
train_df["y_mean_dis_target_y"]=train_df[["y_mean","target_point_y"]].apply(f,axis=1)

# 路径中每个点的x 距离目标x 的距离　（数组）
def f(s):
    points_x = s["points_x"]
    target_point_x =s["target_point_x"]
    ret=[]
    for i in range(0,len(points_x)):
        ret.append(points_x[i]-target_point_x)
    return ret
train_df["x_dis_target_point_x"]=train_df[["points_x","target_point_x"]].apply(f,axis=1)

# 路径中点x 距离目标x 的最小距离，最大距离，等比分点，平均,方差
train_df["x_dis_target_point_x_min"]=train_df.x_dis_target_point_x.apply(lambda x:min(x))
train_df["x_dis_target_point_x_max"]=train_df.x_dis_target_point_x.apply(lambda x:max(x))
train_df["x_dis_target_point_x_25"]=train_df.x_dis_target_point_x.apply(lambda x: np.percentile(x,25))
train_df["x_dis_target_point_x_50"]=train_df.x_dis_target_point_x.apply(lambda x:np.percentile(x,50))
train_df["x_dis_target_point_x_75"]=train_df.x_dis_target_point_x.apply(lambda x:np.percentile(x,75))
train_df["x_dis_target_point_x_average"]=train_df.x_dis_target_point_x.apply(lambda x:np.mean(x))
train_df["x_dis_target_point_x_var"]=train_df.x_dis_target_point_x.apply(lambda x:np.var(x))

# 路径中每个点的y 距离目标y的 距离 （数组）
def f(s):
    points_y = s["points_y"]
    target_point_y = s["target_point_y"]
    ret = []
    for i in range(0,len(points_y)):
        ret.append(points_y[i]-target_point_y)
    return ret
train_df["y_dis_target_point_y"]=train_df[["points_y","target_point_y"]].apply(f,axis=1)

# 路径中点y 距离目标y 的最小距离，最大距离，等比分点，平均,方差
train_df["y_dis_target_point_y_min"]=train_df.y_dis_target_point_y.apply(lambda x:min(x))
train_df["y_dis_target_point_y_max"]=train_df.y_dis_target_point_y.apply(lambda x:max(x))
train_df["y_dis_target_point_y_25"]=train_df.y_dis_target_point_y.apply(lambda x: np.percentile(x,25))
train_df["y_dis_target_point_y_50"]=train_df.y_dis_target_point_y.apply(lambda x:np.percentile(x,50))
train_df["y_dis_target_point_x_75"]=train_df.y_dis_target_point_y.apply(lambda x:np.percentile(x,75))
train_df["y_dis_target_point_y_average"]=train_df.y_dis_target_point_y.apply(lambda x:np.mean(x))
train_df["y_dis_target_point_y_var"]=train_df.y_dis_target_point_y.apply(lambda x:np.var(x))

# y 突变的次数 (台阶个数)
def f(s):
    ret =0
    for i in range(1,len(s)):
        if abs(s[i]-s[i-1])>=2:
            ret+=1
    return ret
train_df["y_diff_count"]=train_df["points_y"].apply(f)

# x多少个不变（每个台阶的长度,数组）
def f(s):
    ret = []
    count =1
    for i  in range(1,len(s)):
        if(s[i]==s[i-1]):
            count+=1
        else:
            ret.append(count)
            count=1
    ret.append(count)

    return ret
    
train_df["x_phases_length"]=train_df["points_y"].apply(f)

# x_phases_length 的min max,25,50,75, mean,var,count
train_df["x_phase_length_min"]=train_df["x_phases_length"].apply(lambda x:min(x))
train_df["x_phase_length_max"]=train_df["x_phases_length"].apply(lambda x:max(x))
train_df["x_phase_length_25"]=train_df["x_phases_length"].apply(lambda x:np.percentile(x,25))
train_df["x_phase_length_50"]=train_df["x_phases_length"].apply(lambda x:np.percentile(x,50))
train_df["x_phase_length_75"]=train_df["x_phases_length"].apply(lambda x:np.percentile(x,75))
train_df["x_phase_length_mean"]=train_df["x_phases_length"].apply(lambda x:np.mean(x))
train_df["x_phase_length_var"]=train_df["x_phases_length"].apply(lambda x:np.var(x))
train_df["x_phase_length_count"]=train_df["x_phases_length"].apply(lambda x:len(x))


train_df[:10]

,points_x,points_y,points_time,target_point_x,target_point_y,target_point,label,velocity_x,velocity_y,accelerate_x,...,x_phase_length_min,x_phase_length_max,x_phase_length_25,x_phase_length_50,x_phase_length_75,x_phase_length_mean,x_phase_length_var,x_phase_length_count,points_y_var,points_x_var
id,,,,,,,,,,,,,,,,,,,,,
1,"[353, 367, 388, 416, 500, 584, 675, 724, 780, ...","[2607, 2607, 2620, 2620, 2620, 2620, 2620, 262...","[349, 376, 418, 442, 493, 547, 592, 643, 694, ...",1420.5,202.0,"1420.5,202",1,"[0.518518518519, 0.5, 1.16666666667, 1.6470588...","[0.0, 0.309523809524, 0.0, 0.0, 0.0, 0.0, 0.0,...","[-0.000536768652711, 0.020202020202, 0.0128104...",...,1,15,2.00,2.0,7.0,5.090909,20.628099,11,377.609375,175323.359375
2,"[283, 290, 297, 304, 311, 325, 339, 346, 360, ...","[2490, 2490, 2490, 2490, 2490, 2503, 2503, 250...","[190, 229, 253, 295, 340, 382, 436, 484, 538, ...",1018.0,189.0,"1018.0,189",1,"[0.179487179487, 0.291666666667, 0.16666666666...","[0.0, 0.0, 0.0, 0.0, 0.309523809524, 0.0, 0.0,...","[0.00356125356125, -0.00378787878788, -0.00025...",...,5,63,19.50,34.0,48.5,34.000000,841.000000,2,11.512760,97218.797578
3,"[255, 297, 353, 423, 479, 542, 605, 668, 745, ...","[2503, 2503, 2529, 2542, 2555, 2568, 2581, 260...","[424, 445, 469, 499, 517, 541, 562, 589, 610, ...",601.5,559.5,"601.5,559.5",1,"[2.0, 2.33333333333, 2.33333333333, 3.11111111...","[0.0, 1.08333333333, 0.433333333333, 0.7222222...","[0.0148148148148, 0.0, 0.0324074074074, -0.023...",...,1,23,1.00,1.0,2.0,3.160000,31.734400,25,4654.220798,60832.483256
4,"[437, 549, 605, 661, 731, 780, 822, 871, 885, ...","[2724, 2711, 2724, 2750, 2789, 2802, 2828, 284...","[466, 514, 553, 613, 730, 751, 787, 847, 907, ...",675.0,202.0,"675.0,202",1,"[2.33333333333, 1.4358974359, 0.933333333333, ...","[-0.270833333333, 0.333333333333, 0.4333333333...","[-0.0206307102859, -0.0101528101528, -0.003785...",...,1,20,1.00,1.0,6.0,3.875000,23.984375,16,1023.496358,17590.502862
5,"[248, 255, 297, 339, 451, 577, 717, 885, 948, ...","[2542, 2542, 2542, 2555, 2568, 2594, 2633, 265...","[73, 109, 163, 217, 265, 316, 364, 424, 466, 5...",787.0,189.0,"787.0,189",1,"[0.194444444444, 0.777777777778, 0.77777777777...","[0.0, 0.0, 0.240740740741, 0.270833333333, 0.5...","[0.012962962963, 0.0, 0.0305010893246, 0.00277...",...,1,25,1.00,1.5,3.0,3.178571,25.075255,28,8561.799015,208168.306527
6,"[171, 178, 178, 206, 353, 486, 668, 815, 899, ...","[2685, 2685, 2672, 2659, 2594, 2594, 2594, 259...","[229, 277, 301, 346, 400, 448, 505, 550, 604, ...",1049.5,1313.5,"1049.5,1313.5",1,"[0.145833333333, 0.0, 0.622222222222, 2.722222...","[0.0, -0.541666666667, -0.288888888889, -1.203...","[-0.00405092592593, 0.0180354267311, 0.0424242...",...,1,17,1.00,1.0,2.0,2.769231,17.715976,13,2611.545525,129859.489969
7,"[227, 234, 241, 248, 255, 262, 269, 276, 283, ...","[2464, 2464, 2464, 2464, 2464, 2464, 2464, 246...","[289, 349, 385, 412, 445, 481, 493, 553, 589, ...",1060.0,748.0,"1060.0,748",1,"[0.116666666667, 0.194444444444, 0.25925925925...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.00162037037037, 0.00205761316872, -0.001571...",...,2,33,7.75,8.5,13.5,12.666667,86.555556,12,654.341023,134735.612188
8,"[199, 241, 255, 283, 339, 395, 465, 493, 507, ...","[2516, 2542, 2542, 2542, 2542, 2542, 2542, 254...","[253, 277, 301, 346, 397, 442, 493, 541, 592, ...",573.5,189.0,"573.5,189",1,"[1.75, 0.583333333333, 0.622222222222, 1.09803...","[1.08333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.0486111111111, 0.00112721417069, 0.0099128...",...,1,16,1.00,2.0,9.0,5.800000,34.960000,5,361.310345,24444.183115
9,"[178, 192, 199, 206, 220, 227, 241, 255, 262, ...","[2503, 2503, 2503, 2503, 2503, 2503, 2503, 250...","[577, 592, 601, 619, 631, 637, 664, 679, 685, ...",724.0,189.0,"724.0,189",1,"[0.933333333333, 0.777777777778, 0.38888888888...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[-0.012962962963, -0.0288065843621, 0.05185185...",...,1,17,2.00,4.0,7.0,6.333333,32.888889,9,1014.4

In [37]:
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn import cross_validation, metrics
train_x= train_df[["points_y_var","points_x_var","velocity_x_mean","velocity_y_mean","accelerate_x_mean","accelerate_y_mean","velocity_x_var","velocity_y_var","accelerate_x_var","accelerate_y_var",
                            "max_velocity_x","max_velocity_y","max_accelerate_x","max_accelerate_y","min_velocity_x","min_velocity_y","min_accelerate_x","min_accelerate_y","count_velocity","velocity_x_percentile_25",
                           "velocity_y_percentile_25", "accelerate_x_percentile_25","accelerate_y_percentile_25","velocity_x_percentile_75","velocity_y_percentile_75","accelerate_x_percentile_75","accelerate_y_percentile_75",
                            "velocity_x_percentile_50","velocity_y_percentile_50","accelerate_x_percentile_50","accelerate_y_percentile_50","x_mean_dis_target_x","y_mean_dis_target_y","x_dis_target_point_x","x_dis_target_point_x_min",
                            "x_dis_target_point_x_max","x_dis_target_point_x_25","x_dis_target_point_x_50","x_dis_target_point_x_75","x_dis_target_point_x_average","x_dis_target_point_x_var","y_dis_target_point_y_min","y_dis_target_point_y_max",
                            "y_dis_target_point_y_25","y_dis_target_point_y_50","y_dis_target_point_x_75","y_dis_target_point_y_average","y_dis_target_point_y_var","y_diff_count","x_phase_length_min","x_phase_length_max","x_phase_length_25",
                           "x_phase_length_50", "x_phase_length_75","x_phase_length_mean","x_phase_length_var","x_phase_length_count"]]
train_y =train_df[["label"]]
x_train, x_test, y_train, y_test = train_test_split(train_x, train_y, test_size = 0.2)
 
person_indices=y_train[y_train.label==1].index
robort_indices=y_train[y_train.label==0].index
person_len = len(robort_indices)* 3
person_random_indices = np.random.choice(person_indices, person_len, replace=False)

x_train1 = pd.concat([x_train.loc[person_random_indices], x_train.loc[robort_indices]])
y_train1= pd.concat([ y_train.loc[person_random_indices], y_train.loc[robort_indices]])

'''
max_depth ：限定了决策树的最大深度，对于防止过拟合非常有用。
min_samples_leaf ：限定了叶子节点包含的最小样本数，这个属性对于防止上文讲到的数据碎片问题很有作用。 (因为随着树的深度增加，叶子上的样本数会越来越少)
善用min_samples_split和min_samples_leaf参数来控制叶子节点的样本数量，防止overfitting。
平衡训练数据中的各个种类的数据，防止被一个种类的数据支配。


http://www.cnblogs.com/pinard/p/6160412.html
内部节点再划分所需最小样本数min_samples_split: 这个值限制了子树继续划分的条件，如果某节点的样本数少于min_samples_split，则不会继续再尝试选择最优特征来进行划分。 默认是2.如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。

叶子节点最少样本数min_samples_leaf: 这个值限制了叶子节点最少的样本数，如果某叶子节点数目小于样本数，则会和兄弟节点一起被剪枝。 默认是1,可以输入最少的样本数的整数，或者最少样本数占样本总数的百分比。如果样本量不大，不需要管这个值。如果样本量数量级非常大，则推荐增大这个值。
'''
param1 = {'n_estimators':range(10,81,10),'max_depth':range(1,101,10)}
gv = GridSearchCV(estimator = RandomForestClassifier(max_features='sqrt' ,random_state=10), 
                       param_grid = param1, scoring='f1',cv=5)
gv.fit(x_train1,y_train1)
gv.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

#clf.feature_importances_


      points_y_var   points_x_var  velocity_x_mean  velocity_y_mean  \
id                                                                    
1414      0.000000  137645.654321         0.747914         0.000000   
1993   3462.849401   33406.081916         0.135162        -0.021929   
1277   1738.576020  150795.893861         0.544018         0.087209   
2432   3045.672099  210916.471111         0.490733         0.083925   
1518  11028.640000   45752.268402         0.315392        -0.239735   
787    1181.715600  150001.250000         0.424450         0.031322   
558    1435.745768  102052.597107         1.484821        -0.200699   
2468   1675.241023   28236.443551         0.670743         0.205015   
1350    188.882353   98943.885813         0.641718        -0.022068   
2183   1905.650731   11204.525952         0.073600         0.007140   
653    2430.977072   34174.320988         0.325029         0.087627   
133    3633.886052   93769.065305         0.723512         0.147296   
2550  